# Anime Recommendation System
#### Alejandra Avitia-Davila, Crystal Feng, Simon Chau, and Tyler Lafond

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

animes_df = pd.read_csv("animes.csv")
profiles_df = pd.read_csv("profiles.csv")

print(animes_df.head())
print()